# Classificação de texto com BERT

## O que é classificação de texto?

Classificação de texto é um subcampo de aprendizado de máquina que ensina computadores a classificar texto em diferentes categorias. É comumente usado como uma técnica de aprendizado supervisionado, o que significa que o algoritmo é treinado em um conjunto de textos que já foram rotulados com suas respectivas categorias. Uma vez treinado nesses dados, o algoritmo pode usar o que aprendeu para fazer previsões sobre novos textos não rotulados.

O algoritmo procura padrões no texto para determinar a qual categoria ele pertence. É como quando aprendemos a reconhecer um certo tipo de flor — começamos a notar certas características que a distinguem de outros tipos de flores. Com a classificação de texto, o algoritmo está fazendo a mesma coisa, mas com palavras e frases.

A classificação de texto é uma ferramenta versátil amplamente usada em muitas aplicações do mundo real que você pode ter encontrado. Por exemplo, um e-mail que acabou na sua pasta de spam é a classificação de texto em ação. O modelo pode diferenciar entre e-mails de spam e não spam estudando palavras ou frases específicas que identificam um determinado e-mail como spam, como "Parabéns, você ganhou" ou "Hoje é seu dia de sorte".

A classificação de texto também é valiosa na análise do sentimento de postagens de mídia social, particularmente quando se trata de detectar sentimentos negativos como discurso de ódio. Ao usar um modelo de aprendizado de máquina, o texto pode ser classificado e monitorado para linguagem ofensiva e discurso de ódio.

Mas a classificação de texto não serve apenas para aplicações sérias — ela também pode ser usada para coisas divertidas, como categorizar artigos de notícias e vídeos por tópico. Dessa forma, um usuário pode facilmente encontrar artigos e vídeos que lhe interessam sem ter que peneirar conteúdo irrelevante. A classificação de texto é realmente uma ferramenta poderosa com uma variedade de aplicações práticas.

## O que é BERT para aprendizado profundo?

BERT, abreviação de Bidirectional Encoder Representations from Transformers, é um poderoso modelo de processamento de linguagem natural (PLN) desenvolvido pelo Google que usa uma arquitetura de rede neural profunda baseada no modelo de transformador de última geração.

Como dissemos antes, a arquitetura do modelo BERT é baseada em uma rede neural profunda chamada transformador, que é diferente dos modelos NLP tradicionais que processam texto uma palavra por vez. Em vez disso, os transformadores podem processar toda a entrada de texto de uma só vez, o que os ajuda a capturar as relações entre palavras e frases de forma mais eficaz.

## Como o modelo BERT funciona para classificação de texto?

O BERT usa um codificador transformador bidirecional multicamadas para representar o texto de entrada em um espaço de alta dimensão. Isso significa que ele pode levar em conta todo o contexto de cada palavra na frase, o que o ajuda a entender melhor o significado do texto.

Uma das coisas mais interessantes sobre o BERT é que ele é um modelo pré-treinado. Isso significa que o BERT pode ser treinado em grandes quantidades de dados de texto, como livros, artigos e sites, antes de ser ajustado para tarefas específicas de PNL downstream, incluindo classificação de texto.

Ao pré-treinar em um grande corpus de dados de texto, o BERT pode desenvolver uma compreensão profunda da estrutura e do significado subjacentes da linguagem, tornando-o uma ferramenta altamente eficaz para tarefas de PNL. Uma vez pré-treinado, o BERT pode ser ajustado para tarefas específicas, o que permite que ele se adapte às nuances específicas da tarefa e melhore sua precisão.

É importante notar que há duas variações diferentes do BERT: BERT base e BERT grande. Para o restante deste artigo, utilizaremos o modelo base do BERT, que é uma versão mais compacta do BERT que ainda mantém uma forte compreensão do contexto e das nuances linguísticas. Ao empregar o BERT base, podemos aproveitar suas habilidades enquanto reduzimos as demandas computacionais, tornando-o mais adequado para uma variedade de tarefas de classificação de texto e alcançando resultados notáveis.

## Tutorial sobre classificação de texto usando BERT

Então por que ajustamos o BERT no conjunto de dados de resenhas de filmes do IMDB? Bem, queremos adaptar o já poderoso modelo BERT para tarefas de análise de sentimentos. O BERT é excelente para entender a estrutura e o contexto da linguagem, mas não tem habilidades naturais de análise de sentimentos. Ao ajustar o BERT para classificação de texto com um conjunto de dados rotulado, como resenhas de filmes do IMDB, damos a ele a capacidade de prever com precisão os sentimentos nas frases que ele encontra.

Caso você queira executar o modelo abaixo, você pode encontrar o conjunto de dados IMDB no seguinte link:

[Conjunto de dados do IMDB com 50 mil críticas de filmes](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews).

## Etapa 1: Importe as bibliotecas necessárias

Este trecho de código é sobre importar as ferramentas essenciais que precisamos para nosso projeto. Estamos usando PyTorch para a funcionalidade de aprendizado profundo, a biblioteca transformers para BERT e métodos essenciais da biblioteca scikit-learn para manipular dados e verificar o quão bem nosso modelo funciona.

In [ ]:
!pip install transformers
!pip install scikit-learn 
!pip install pandas
!pip install accelerate
!pip install bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

## Etapa 2: Importe o conjunto de dados IMDB e pré-processe-o

O código abaixo define uma função load_imdb_data que lê um arquivo CSV contendo resenhas de filmes do IMDB e seus sentimentos correspondentes. Ele retorna uma lista de textos de resenhas e uma lista de rótulos, onde 1 representa um sentimento positivo e 0 representa um sentimento negativo.

In [ ]:
def load_imdb_data(data_file):
    df = pd.read_csv(data_file)
    texts = df['review'].tolist()
    labels = [1 if sentiment == "positive" else 0 for sentiment in df['sentiment'].tolist()]
    return texts, labels

Em seguida, salvaremos nosso diretório de conjunto de dados e o inseriremos como uma entrada na função load_imdb_data().

In [ ]:
data_file = "../dataset/IMDB Dataset.csv"
texts, labels = load_imdb_data(data_file)

## Etapa 3: Crie uma classe de conjunto de dados personalizada para classificação de texto

Esta é uma classe de conjunto de dados personalizado que ajuda a organizar as avaliações de filmes e seus sentimentos para nosso modelo BERT. Ela cuida da tokenização do texto, manipulando o comprimento da sequência e fornecendo um pacote organizado com IDs de entrada, máscaras de atenção e rótulos para nosso modelo aprender.

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}


## Etapa 4: construir nosso classificador BERT do cliente

Nesta etapa, pretendemos criar nosso próprio classificador BERT personalizado. O classificador é construído sobre o famoso modelo BERT, que é ótimo para entender texto. Adicionaremos então uma camada de dropout para manter as coisas sob controle e uma camada linear para nos ajudar a classificar o texto.

Nosso BERTClassifier pega alguns IDs de entrada e máscaras de atenção, e os executa por meio do BERT e das camadas extras que adicionamos. O classificador retorna nossa saída como pontuações de classe.

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

## Etapa 5: Defina a função train()

A função train() pega o modelo, o carregador de dados, o otimizador, o planejador e o dispositivo como seus trainees. A função coloca o modelo no modo de treinamento e então executa cada lote de dados do carregador de dados. Para cada lote, ela limpa os gradientes do otimizador, obtém os IDs de entrada, máscaras de atenção e rótulos, e os alimenta para o modelo.

In [ ]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

## Etapa 6: Construir nosso método de avaliação

In [ ]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

## Etapa 7: Defina os parâmetros do nosso modelo

Aqui, vamos configurar parâmetros essenciais para ajustar o BERTClassifier, incluindo o nome do modelo BERT, número de classes, comprimento máximo da sequência de entrada, tamanho do lote, número de períodos de treinamento e taxa de aprendizado, para ajudar o modelo a entender efetivamente as críticas de filmes e seus sentimentos.

In [ ]:
bert_model_name = "google-bert/bert-base-uncased"
num_classes = 2
max_length = 64
batch_size = 8     
num_epochs = 4
learning_rate = 2e-5

## Etapa 8: Carregando e dividindo os dados.

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

## Etapa 9: inicializar o tokenizador, o conjunto de dados e o carregador de dados

Para Windows na classe DataLoader o num_workers=0

In [ ]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

## Etapa 10: Configurar o dispositivo e o modelo

In [ ]:
print(torch.cuda.is_available())
print(torch.version.cuda)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)
print(device)

## Etapa 11: Configurar o otimizador e o planejador de taxa de aprendizagem

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

## Etapa 12: Treinando o modelo

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

Salvando o modelo final



In [ ]:
os.makedirs("../model", exist_ok=True)
torch.save(model.state_dict(), "../model/bert_classifier.bin")

## Etapa 13: Construa nosso método de previsão

A função predict_sentiment() atua como nosso método de avaliação. Para cada lote, ela obtém os IDs de entrada, máscaras de atenção e rótulos e os alimenta para o modelo. O modelo então fornece suas melhores previsões, que são comparadas aos rótulos reais.

Por fim, a função calcula a pontuação de precisão e um relatório de classificação para nos informar o quão bem o modelo se saiu na compreensão dos sentimentos das críticas de filmes.

In [ ]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        
    return "positive" if preds.item() == 1 else "negative"

## Etapa 14: Avaliando o desempenho do nosso modelo

In [ ]:
test_text = "The movie is very good and I recommend it to everyone."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("The movie is very good and I recommend it to everyone.")
print(f"Predicted sentiment: {sentiment}")

Resultado: O filme é muito bom e eu recomendaria a todos.

Sentimento previsto: positivo

In [ ]:
test_text = "Worst movie of the year."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("Worst movie of the year.")
print(f"Predicted sentiment: {sentiment}")

Resultado: Pior filme do ano.

Sentimento previsto: negativo



## Considerações finais sobre a classificação de texto usando BERT

Para resumir, o BERT mudou seriamente o jogo quando se trata de classificação de texto. Ele tornou coisas como análise de sentimentos e classificação de tópicos muito melhores e mais rápidas. Ao pegar esses modelos pré-treinados e personalizá-los para nossos próprios projetos, estamos obtendo resultados incríveis que nos ajudam no mundo real.

À medida que continuamos experimentando o BERT e modelos similares, não há dúvidas de que veremos ainda mais coisas legais acontecendo no mundo da IA ​​e da compreensão de linguagem.